### Introduction
Climbing has relatively recently picked up in popularity, being included for the first time in the Tokyo 2020 olympics. There is very little data on climbing habits of successful (and unsucessful) climbers, and training routines are generally heavily anecdotal. A reddit user, u/higiff, posted a survey on the "climb harder" subreddit (https://www.reddit.com/r/climbharder/comments/6693ua/climbharder_survey_results/), in order to question climbers about their training habits. This to my knowledge is the largest publically available dataset on climbers and their training strategies.

Having been climbing consistently for around four years now, I was intrigued to see what information this dataset might contain. Causal inference is unfortunately impossible with this dataset, but nevertheless, in the absence of better data, understanding the habits of highly effective climbers is still potentially useful.

I have cleaned the data and extracted features in the notebook data_cleaning.ipynb. Here, I will deal with missing values and build a model to try and predict `max_boulder_grade` from features such as training strategies, height, sex etc.

### Imports
All imports are included here:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import joblib

### Data imputation
The remaining missing data seems to be missing completely at random.

I'll try two strategies and see which offers the best results:

**1. No data imputation by dropping rows/columns with `np.nan` :**
- Advantage: Avoids potential bias from imputing data. Gives a good benchmark.
- Disadvantage: Less training data, loss of power.

**2. Imputing missing values in `X_train` and `X_test`:**
- Advantage: Potentially more power as can utilise the train set more fully.
- Disadvantage: Imputation can often bias the model and make it perform worse.

In [2]:
np.random.seed(1)
df = pd.read_csv("cleaned_data.csv")

for col in list(df):
    missing = df[col].isnull().sum()
    if missing > 0:
        print("{} has {} missing values".format(col, missing))

arm_span_cm has 33 missing values
max_boulder_grade has 17 missing values
max_pull_ups has 120 missing values


#### 1. No data imputation by dropping rows/columns with `np.nan`

In [3]:
df_drop_na = df.copy()
df_drop_na = df_drop_na.drop(columns=["max_pull_ups", "arm_span_cm"]) #  Drop cols with lots of np.nans
df_drop_na = df_drop_na.dropna() #  Drop remaining rows with nas

print("Dataset has shape: {}".format(df_drop_na.shape))
print("{} np.nan values in dataset".format(df_drop_na.isna().values.sum()))

Dataset has shape: (520, 49)
0 np.nan values in dataset


I'll train a random forest regressor, and use 5-fold cross validation and assess the model fit with the R^2 value.

In [4]:
X = df_drop_na.drop(columns="max_boulder_grade")
y = df_drop_na["max_boulder_grade"]
rf = RandomForestRegressor()  # Use defualts for now
scores = cross_val_score(rf, X, y, cv=5, scoring="r2")
print("R^2: {:.3f} +/- {:.3f}".format(scores.mean(), scores.std() * 2))

R^2: 0.501 +/- 0.171


#### 2. Imputing np.nan in `X_train` and `X_test`:
Let's see if we can improve this R^2 score by imputing the missing values instead. I'll use the `IterarativeImputer()` function from scikit-learn. This will (by defualt) impute values sequentially using Bayesian ridge regression.

In [5]:
df_impute = df.copy()
df_impute = df_impute[df_impute['max_boulder_grade'].notna()] # Keep all na rows except in our dependant variable
print(df_impute.shape)

(520, 51)


In [6]:
X = df_impute.drop(columns = "max_boulder_grade")
y = df_impute["max_boulder_grade"]

pipeline = Pipeline([
    ("imputer", IterativeImputer(max_iter=100)),
    ("rf", RandomForestRegressor())
])

scores = cross_val_score(pipeline, X, y, cv=5, scoring="r2")
print("R^2: {:.3f} +/- {:.3}".format(scores.mean(), scores.std() * 2))

R^2: 0.531 +/- 0.152


Imputation increased the R^2 value by ~0.3!

### Grid search for hyperparmater optimization

In [7]:
X = df_impute.drop(columns="max_boulder_grade")
y = df_impute["max_boulder_grade"]

pipeline = Pipeline([
    ("imputer", IterativeImputer(max_iter=100)),
    ("rf", RandomForestRegressor())
])

param_grid = {
    "imputer__initial_strategy": ["mean", "median"],
    "rf__n_estimators": [100, 500, 1000],
}

grid = GridSearchCV(pipeline, cv=5, param_grid=param_grid, scoring="r2")
grid.fit(X, y)

std = grid.cv_results_["std_test_score"][grid.cv_results_["rank_test_score"]==1][0]
print("Best score:")
print("R2: {:.3f} +/- {:.3}\n".format(grid.best_score_, std*2))
print("Achieved with hyperparameters:\n{}".format(grid.best_params_))

Best score:
R2: 0.533 +/- 0.15

Achieved with hyperparameters:
{'imputer__initial_strategy': 'median', 'rf__n_estimators': 1000}


### Save model

In [8]:
joblib.dump(grid.best_estimator_, 'rf_model.joblib', compress = 1)

['rf_model.joblib']

### To do
- Add a proper EDA.
- Try other regressors and compare performance.
- Look at importances of features in the model.